开始数据清理！

In [1]:
pip install cn2an

     |████████████████████████████████| 224 kB 1.5 MB/s            
     |████████████████████████████████| 137 kB 17.2 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import scipy
from collections import Counter
import re
from sklearn.cluster import KMeans
from cn2an import cn2an

In [3]:
#将训练集和测试集合并，方便进行同样的数据预处理
trainData = pd.read_csv("/home/mw/input/quant4533/ruc_Class25Q1_train.csv")
testData = pd.read_csv("/home/mw/input/quant4533/ruc_Class25Q1_test.csv")
fullData = pd.concat([trainData,testData],axis=0)

In [4]:
trainData.shape
#可知前多少行是train

(84133, 31)

In [5]:
fullData.head()

,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,建筑面积,套内面积,...,抵押信息,房屋优势,核心卖点,户型介绍,周边配套,交通出行,lon,lat,年份,ID
0,0,79.0,111.0,二至三环,人定湖西里,6564200.0,2室1厅1厨1卫,中楼层 (共5层),52.3㎡,NaN,...,NaN,装修、房本满五年,此房是南北通透小板楼，户型方正，格局合理,房子是南北通透户型方正采光好，前后没有遮挡视野好，通风效果好,医院、公园、超市，生活便利，火箭军医院、积水潭医院，双秀公园，人定湖公园，物美超市、世纪华联等。,NaN,116.389326,39.963727,2018.0,NaN
1,0,43.0,231.0,五至六环,龙跃苑四区,4174000.0,3室1厅1厨1卫,顶层 (共6层),127.44㎡,123.7㎡,...,NaN,装修、房本满五年,南北通透商品房自住装修无个税,房子三居一卫，户型方正，南北通透，客厅朝南带阳台，主卧朝南，东向有窗户，次卧、厨房朝北，厨房...,医院：北京京都儿童医院、昌平中西结合医院，积水潭医院配套设施：美廉美，工商，农行，邮政等银行...,NaN,116.354287,40.079237,2017.0,NaN
2,0,97.0,54.0,五至六环,名都园,16310000.0,4室2厅1厨4卫,底层 (共3层),228.54㎡,NaN,...,NaN,房本满五年,有挑空客厅私家车库花园和*,地上三层地下一层四期密度低没有公寓一层车库可以做卧室花园有100平米左右,NaN,NaN,116.543168,40.078165,2018.0,NaN
3,0,62.0,568.0,三至四环,保利海德公园,2834600.0,2房间2卫,低楼层 (共10层),43.6㎡,29.39㎡,...,NaN,地铁、房本满五年、,NaN,NaN,NaN,NaN,116.357585,39.980030,2016.0,NaN
4,0,62.0,226.0,三至四环,京投银泰琨御府,1954000.0,1房间1卫,中楼层 (共10层),39.85㎡,29.94㎡,...,NaN,地铁、房本满五年,此房朝向是南向户型，小区位于地铁6号线和10号线的*,NaN,NaN,小区为一站式地铁，楼栋位于*，地铁线路为6号线和10号线慈寿寺站，位于上盖下边。,116.299697,39.940604,2017.0,NaN


In [6]:
fullData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98919 entries, 0 to 14785
Data columns (total 32 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   城市      98919 non-null  int64  
 1   区域      98919 non-null  float64
 2   板块      98919 non-null  float64
 3   环线      52054 non-null  object 
 4   小区名称    98919 non-null  object 
 5   价格      84133 non-null  float64
 6   房屋户型    98312 non-null  object 
 7   所在楼层    98919 non-null  object 
 8   建筑面积    98919 non-null  object 
 9   套内面积    29453 non-null  object 
 10  房屋朝向    98919 non-null  object 
 11  建筑结构    98312 non-null  object 
 12  装修情况    98312 non-null  object 
 13  梯户比例    96942 non-null  object 
 14  配备电梯    89306 non-null  object 
 15  别墅类型    795 non-null    object 
 16  交易时间    98919 non-null  object 
 17  交易权属    98919 non-null  object 
 18  上次交易    68312 non-null  object 
 19  房屋用途    98917 non-null  object 
 20  房屋年限    67393 non-null  object 
 21  产权所属    98919 non-null  object 
 22

思路：  
1.**城市、区域、板块**的号码没有数值大小含义，后续需要转变成哑变量。区域和板块很多，需要思考处理方法。  
2.**环线**：用匹配小区的填写空值，如果仍没有，填写“无”  
3.**小区名称**需要和补充的数据结合，生成更多特征，并补充特征。  
4.**价格**：log之后再转回来  
5.**房屋户型**拆开  
6.**所在楼层**：拆成多列：全部楼层、所在楼层、楼层比例  
7.**建筑面积**：转换类型；套内面积空值太多，丢弃  
8.处理之后dummy：**建筑结构、装修情况、配备电梯、别墅类型、房屋用途、房屋年限、产权所属、年份**  
9.**房屋朝向**：分开之后dummy  
10.**电梯处理**：一户平均+有无。后续添加交互项（和楼层）  
11.**交易时间和上次交易转换成时间类型之后相减**，可以获得新特征：上次交易间隔  
12.**房屋优势**可以拆成三列  
13.**核心卖点、户型介绍、周边配套、交通出行**可以计算高频词出现次数  
14.先用小区名称匹配，之后使用**经纬度**最近匹配剩下的。  


In [5]:
#删除全空的“抵押信息”列
fullData = fullData.drop(columns=['抵押信息'])

In [6]:
#把变建筑面积、套内面积转为float
fullData['建筑面积'] = fullData['建筑面积'].apply(lambda x:x[:-1]).astype('float')
fullData['套内面积'] = fullData['套内面积'].apply(lambda x: float(x[:-1]) if pd.notna(x) else x)
#把城市、区域、板块转为str
fullData = fullData.astype({'城市':'str','区域':'str','板块':'str'})

In [7]:
fullData.describe()

,价格,建筑面积,套内面积,lon,lat,年份,ID
count,8.413300e+04,98919.000000,29453.000000,98919.000000,98919.000000,98919.000000,14786.000000
mean,1.971953e+06,96.058077,81.528711,113.678653,35.151817,2021.209778,7392.500000
std,2.639962e+06,60.426296,43.454421,6.634010,5.508274,0.879562,4268.494875
min,7.828000e+04,10.000000,1.000000,106.197420,29.258156,2015.000000,0.000000
25%,6.974000e+05,66.180000,57.430000,106.606972,29.717963,2021.000000,3696.250000
50%,1.146500e+06,88.760000,73.610000,114.276243,34.221395,2021.000000,7392.500000
75%,2.176000e+06,115.000000,97.530000,116.678048,39.920243,2022.000000,11088.750000
max,7.995000e+07,10337.000000,960.490000,129.581642,46.340684,2023.000000,14785.000000


## 查看分布情况

In [10]:
def Box_qq_plots(df):
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))

    # 绘制箱线图
    sns.boxplot(data=df, ax=axes[0])
    axes[0].set_title("Box Plot")

    # 绘制 QQ 图
    qqplot(df, line='s', ax=axes[1])
    axes[1].set_title("Normal QQPlot")

    plt.show()

In [18]:
Box_qq_plots(trainData['价格'])

<Figure size 1008x432 with 2 Axes>

In [19]:
plt.figure(figsize=(9, 8))
sns.distplot(trainData['价格'], color='g', bins=100, hist_kws={'alpha': 0.4})

/opt/conda/lib/python3.7/site-packages/seaborn/distributions.py:2619: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:xlabel='价格', ylabel='Density'>

<Figure size 648x576 with 1 Axes>

In [19]:
#缩尾处理
#fullData['价格'][:84133] = mstats.winsorize(fullData['价格'][:84133], limits=[0.005, 0.005])

#boxcox处理，需要是正态分布，但从图表可知price不太像正态分布。效果可能不是很好。
#train['Rent Index'],fitted_lambda= scipy.stats.boxcox(train['Rent Index'] ,lmbda=None)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
#跑不出来，可能是因为数据量太大一直在转圈
Box_qq_plots(fullData['建筑面积'].loc[:84132])

<Figure size 1008x432 with 2 Axes>

## 匹配小区数据

In [7]:
xiaoqu = pd.read_csv("/home/mw/project/xiaoqu_rent.csv")
xiaoqu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2973 entries, 0 to 2972
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   名称       2973 non-null   object 
 1   环线位置     1730 non-null   object 
 2   建筑年代     2973 non-null   float64
 3   房屋总数     2973 non-null   int64  
 4   楼栋总数     2973 non-null   int64  
 5   物业公司     2344 non-null   object 
 6   绿 化 率    2973 non-null   float64
 7   容 积 率    2973 non-null   float64
 8   物 业 费    2630 non-null   object 
 9   供水       2798 non-null   object 
 10  供暖       2273 non-null   object 
 11  供电       2812 non-null   object 
 12  燃气费      2564 non-null   object 
 13  供热费      1190 non-null   object 
 14  停车位      2973 non-null   float64
 15  停车费用     2973 non-null   float64
 16  coord_x  2973 non-null   float64
 17  coord_y  2973 non-null   float64
 18  匹配租房     2973 non-null   object 
 19  小区名称     2973 non-null   object 
 20  平均租金     2973 non-null   float64
dtypes: float64(8),

In [8]:
is_matching = fullData['小区名称'].isin(xiaoqu['名称'])

# 获取匹配的索引
matching_indexes = fullData.index[is_matching]

print("完全匹配的索引:")
print(matching_indexes)

完全匹配的索引:
Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            14776, 14777, 14778, 14779, 14780, 14781, 14782, 14783, 14784,
            14785],
           dtype='int64', length=92246)


In [9]:
not_matching = ~fullData['小区名称'].isin(xiaoqu['名称'])
# 获取不匹配的索引
unmatching_indexes = fullData.index[not_matching]
print(unmatching_indexes)
# 获取不匹配的小区名称
unmatched_names = fullData.loc[not_matching, '小区名称']
unmatched_names.unique()

Int64Index([ 1099,  1589,  1884,  1971,  1977,  2209,  2521,  2592,  2605,
             2995,
            ...
            14709, 14714, 14733, 14739, 14748, 14754, 14755, 14757, 14769,
            14773],
           dtype='int64', length=6673)


array(['融泽嘉园中路12号院', '首创悦树湾', '耳东文化产业园', '翡翠公园六期', '听竹院', '北京风景66号院',
       '万科长阳天地5号院', '永华道南门外85号办事处宿舍', '雅馨园', '孔雀城翠岚园涵碧园', '潮白河孔雀城紫薇园',
       '运河源建材城', '林城万人温泉小镇二期', '翠湖天地', '美丽蓝湾', '星城汇', '华竹花园', '紫金一品',
       '鲁能星城11街区', '东海假日花园（洋房）', '长寿佳园', '金科中央公园城（璧山）10号门', '融科橡树澜湾B组团',
       '教委集资大院', '长寿桃花园', '奇峰清华源三期', '悦水岸', '龙湖水晶郦城朗苑', '融科金色时代二期',
       '碧桂园东原合能千屿一期', '盛唐叠彩山二期', '金科天宸五街区', '云盛路26号', '月华村', '凌云峰一期',
       '金科御临河星光', '建宇新时区', '北辰悦来壹号璟悦', '世纪金马花漫里', '保利堂悦青竹园', '金九城北区',
       '御璟湖山璟時北', '泽瑞琥珀天成北区', '龙湖千山新屿悦汐', '荣盛观云郡', '中粮中央公园祥云A区', '绿地克拉公馆',
       '龙湖九里峰景', '国宾城御府', '宝桐路205号', '金紫山', '绿城两江御园玉茗园', '华远海蓝和光',
       '万科金开悦府锦悦', '十里长江·澜庭', '富州平顶山', '璞缇1号栖珑', '万科金开悦府风悦', '雅居乐富春山居璟和里',
       '天盈首原Beyond', '天安江畔珑园2期', '金科中央公园城（璧山）1号2号门', '融信海月渝州',
       '桥达天蓬樾府·樱花里', '中海天钻', '青秀阅山', '锦湖路62号', '九龙西苑', '和庆家园C区', '尚美时代B区',
       '龙湖紫云赋', '旭辉金科空港', '翰粼天辰', '云湖绿岛', '石堡街5号', '龙城大院', '荣安九龙壹号',
       '华宇城M8', '万科招商理想城学府', '金辉中央铭著D区', '龙湖西宸原著二三组团', '荣盛华府御珑',
       '裕鸿佳苑(安秀路480

In [10]:
#按照最近的经纬度匹配小区
def haversine(lon1, lat1, lon2, lat2):
    """
    计算两个经纬度点之间的距离（单位：千米）
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [11]:
# 对于匹配的索引的行，直接添加匹配的小区名称
fullData.loc[matching_indexes, '匹配小区'] = fullData.loc[matching_indexes, '小区名称']

# 对于不匹配的索引的行，根据经纬度匹配最近的 xiaoqu 的名称
for index, row in fullData.loc[unmatching_indexes].iterrows():
    distances = haversine(row['lon'], row['lat'], xiaoqu['coord_x'], xiaoqu['coord_y'])
    nearest_index = np.argmin(distances)
    fullData.loc[index, '匹配小区'] = xiaoqu.loc[nearest_index, '名称']

In [12]:
columns_to_merge = ['建筑年代','房屋总数','楼栋总数','绿 化 率', '容 积 率','停车位','停车费用','环线位置','平均租金']
fullData = fullData.merge(xiaoqu[['名称'] + columns_to_merge],
                          left_on='匹配小区', right_on='名称', how='left')

#补充环线信息
fullData['环线'] = fullData['环线'].combine_first(fullData['环线'])
fullData['环线'].fillna("无", inplace=True)

# 删除多余的“名称”列
fullData = fullData.drop(columns=['名称', '环线位置'])
fullData.head()

,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,建筑面积,套内面积,...,ID,匹配小区,建筑年代,房屋总数,楼栋总数,绿 化 率,容 积 率,停车位,停车费用,平均租金
0,0,79.0,111.0,二至三环,人定湖西里,6564200.0,2室1厅1厨1卫,中楼层 (共5层),52.30,NaN,...,NaN,人定湖西里,1977.5,1317,19,30.0,3.00,300.0,0.0,8660.555556
1,0,43.0,231.0,五至六环,龙跃苑四区,4174000.0,3室1厅1厨1卫,顶层 (共6层),127.44,123.70,...,NaN,龙跃苑四区,2005.0,2317,40,30.0,1.73,1550.0,150.0,7204.461538
2,0,97.0,54.0,五至六环,名都园,16310000.0,4室2厅1厨4卫,底层 (共3层),228.54,NaN,...,NaN,名都园,2001.5,1249,565,30.1,0.64,0.0,0.0,18868.095238
3,0,62.0,568.0,三至四环,保利海德公园,2834600.0,2房间2卫,低楼层 (共10层),43.60,29.39,...,NaN,保利海德公园,2015.0,577,12,40.0,2.60,950.0,1150.0,15446.387097
4,0,62.0,226.0,三至四环,京投银泰琨御府,1954000.0,1房间1卫,中楼层 (共10层),39.85,29.94,...,NaN,京投银泰琨御府,2010.5,1685,19,60.0,1.58,1800.0,1200.0,10737.601626


In [15]:
fullData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98919 entries, 0 to 98918
Data columns (total 40 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   城市      98919 non-null  object 
 1   区域      98919 non-null  object 
 2   板块      98919 non-null  object 
 3   环线      98919 non-null  object 
 4   小区名称    98919 non-null  object 
 5   价格      84133 non-null  float64
 6   房屋户型    98312 non-null  object 
 7   所在楼层    98919 non-null  object 
 8   建筑面积    98919 non-null  float64
 9   套内面积    29453 non-null  float64
 10  房屋朝向    98919 non-null  object 
 11  建筑结构    98312 non-null  object 
 12  装修情况    98312 non-null  object 
 13  梯户比例    96942 non-null  object 
 14  配备电梯    89306 non-null  object 
 15  别墅类型    795 non-null    object 
 16  交易时间    98919 non-null  object 
 17  交易权属    98919 non-null  object 
 18  上次交易    68312 non-null  object 
 19  房屋用途    98917 non-null  object 
 20  房屋年限    67393 non-null  object 
 21  产权所属    98919 non-null  object 
 22

## 房屋户型

In [13]:
pattern = re.compile(r'(\d+)室|(\d+)房间|(\d+)厅|(\d+)厨|(\d+)卫')

# 提取数据并处理
def extract_room_info(row):
    if pd.isna(row):
        return pd.Series([0, 0, 0, 0], index=['房间数量', '客厅数量', '厨房数量', '卫生间数量'])
    matches = pattern.finditer(row)
    room_count, hall_count, kitchen_count, bathroom_count = 0, 0, 0, 0
    for match in matches:
        group = match.group()
        if '室' in group or '房间' in group:
            room_count += int(re.search(r'\d+', group).group())
        elif '厅' in group:
            hall_count += int(re.search(r'\d+', group).group())
        elif '厨' in group:
            kitchen_count += int(re.search(r'\d+', group).group())
        elif '卫' in group:
            bathroom_count += int(re.search(r'\d+', group).group())

    return pd.Series([room_count, hall_count, kitchen_count, bathroom_count], index=['房间数量', '客厅数量', '厨房数量', '卫生间数量'])


In [14]:
# 应用函数并直接获取结果
fullData[['房间数量', '客厅数量', '厨房数量', '卫生间数量']] = fullData['房屋户型'].apply(extract_room_info)

## 所在楼层

In [15]:
def extract_floor_info(row):
    # 提取总楼层数
    total_floor_match = re.search(r'共(\d+)层', row)
    total_floor = int(total_floor_match.group(1)) if total_floor_match else 0

    # 提取实际楼层位置
    if '地下室' in row:
        floor_position = -1
    elif '底层' in row:
        floor_position = 1
    elif '顶层' in row:
        floor_position = total_floor
    elif '低楼层' in row:
        floor_position = int(total_floor * 0.3) if total_floor > 0 else 0
    elif '中楼层' in row:
        floor_position = int(total_floor * 0.6) if total_floor > 0 else 0
    elif '高楼层' in row:
        floor_position = int(total_floor * 0.9) if total_floor > 0 else 0
    else:
        floor_position = 0

    return pd.Series([floor_position, total_floor], index=['实际楼层位置', '总楼层数'])

In [16]:
# 应用函数提取信息
fullData[['实际楼层位置', '总楼层数']] = fullData['所在楼层'].apply(extract_floor_info)

In [17]:
#防止数据泄露，只用train的众数替换
mode_floor = fullData.loc[:84132, '总楼层数'].mode()[0]

fullData['楼层比例'] = fullData['实际楼层位置'] / fullData['总楼层数'].replace(0, mode_floor)

In [18]:
fullData[['楼层比例','总楼层数','实际楼层位置']].head()

,楼层比例,总楼层数,实际楼层位置
0,0.600000,5,3
1,1.000000,6,6
2,0.333333,3,1
3,0.300000,10,3
4,0.600000,10,6


## 处理板块和区域

In [20]:
fullData['板块'].unique()

array(['111.0', '231.0', '54.0', '568.0', '226.0', '616.0', '504.0',
       '592.0', '559.0', '8.0', '678.0', '585.0', '749.0', '686.0',
       '97.0', '161.0', '423.0', '510.0', '201.0', '675.0', '329.0',
       '446.0', '289.0', '298.0', '85.0', '717.0', '768.0', '722.0',
       '182.0', '309.0', '536.0', '347.0', '243.0', '87.0', '66.0',
       '110.0', '1.0', '282.0', '81.0', '137.0', '196.0', '652.0',
       '414.0', '780.0', '59.0', '762.0', '637.0', '795.0', '766.0',
       '132.0', '165.0', '402.0', '588.0', '687.0', '615.0', '145.0',
       '533.0', '136.0', '33.0', '802.0', '589.0', '345.0', '758.0',
       '667.0', '625.0', '262.0', '126.0', '529.0', '260.0', '154.0',
       '374.0', '420.0', '740.0', '305.0', '186.0', '750.0', '403.0',
       '803.0', '445.0', '274.0', '565.0', '334.0', '493.0', '228.0',
       '767.0', '40.0', '419.0', '416.0', '210.0', '514.0', '36.0',
       '761.0', '82.0', '11.0', '412.0', '316.0', '787.0', '300.0',
       '308.0', '655.0', '191.0', '1

In [19]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

In [20]:
fullData['area_block_combined'] = fullData['区域'].astype(str) + '_' + fullData['板块'].astype(str)

# 对合并后的列进行独热编码
encoder = OneHotEncoder(sparse=False)
encoded = encoder.fit_transform(fullData[['area_block_combined']])
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['area_block_combined']))

# 设置聚类数量为 10，比城市数量 6 多一些
n_clusters = 12

# 进行聚类
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(encoded)
fullData['cluster_label'] = kmeans.labels_ + 1

In [21]:
fullData['cluster_label'].unique()

array([ 1, 11,  6,  2, 12,  3, 10,  4,  5,  8,  7,  9], dtype=int32)

## 朝向

In [22]:
orientations = ['东', '南', '西', '北', '东南', '西南', '西北', '东北']

# 进行独热编码
for orientation in orientations:
    fullData[orientation] = fullData['房屋朝向'].apply(lambda x: 1 if orientation in x else 0)

# 查看处理后的数据
fullData.head()

,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,建筑面积,套内面积,...,area_block_combined,cluster_label,东,南,西,北,东南,西南,西北,东北
0,0,79.0,111.0,二至三环,人定湖西里,6564200.0,2室1厅1厨1卫,中楼层 (共5层),52.30,NaN,...,79.0_111.0,1,0,1,0,1,0,0,0,0
1,0,43.0,231.0,五至六环,龙跃苑四区,4174000.0,3室1厅1厨1卫,顶层 (共6层),127.44,123.70,...,43.0_231.0,1,0,1,0,1,0,0,0,0
2,0,97.0,54.0,五至六环,名都园,16310000.0,4室2厅1厨4卫,底层 (共3层),228.54,NaN,...,97.0_54.0,1,0,1,0,0,0,0,0,0
3,0,62.0,568.0,三至四环,保利海德公园,2834600.0,2房间2卫,低楼层 (共10层),43.60,29.39,...,62.0_568.0,1,0,0,0,1,0,0,0,0
4,0,62.0,226.0,三至四环,京投银泰琨御府,1954000.0,1房间1卫,中楼层 (共10层),39.85,29.94,...,62.0_226.0,1,0,1,0,0,0,0,0,0


## 电梯问题

In [24]:
fullData['梯户比例'].unique()

array(['一梯三户', '一梯两户', '一梯一户', '三梯二十三户', '两梯十一户', '六梯十七户', '三梯八户', '一梯四户',
       '两梯二十五户', '两梯三户', '两梯六户', '六梯十六户', nan, '两梯两户', '五梯二十四户', '两梯十九户',
       '六梯十户', '两梯九户', '两梯二十四户', '一梯九户', '两梯十五户', '三梯十户', '两梯八户', '四梯九户',
       '两梯四户', '三梯十二户', '六梯九户', '两梯十户', '一梯七户', '四梯二十七户', '五梯十五户', '五梯九户',
       '两梯十六户', '一梯五户', '两梯二十户', '八梯二十二户', '三梯六户', '四梯四十九户', '三梯九户',
       '一梯八户', '四梯四十七户', '三梯十七户', '三梯二十户', '四梯十六户', '四梯六户', '两梯七户',
       '五梯六十户', '五梯十八户', '两梯十二户', '四梯五十五户', '四梯三十九户', '四梯十一户', '三梯二十二户',
       '两梯十四户', '两梯五户', '三梯二十五户', '一梯二十户', '三梯两户', '一梯十二户', '一梯六户',
       '八梯二十八户', '三梯五十三户', '两梯十七户', '三梯十一户', '一梯二十五户', '三梯七户', '六梯八户',
       '三梯十四户', '两梯二十八户', '三梯三十户', '五梯七户', '五梯二十六户', '四梯八户', '两梯十三户',
       '一梯十五户', '五梯八户', '四梯二十六户', '四梯三十三户', '四梯五户', '四梯十九户', '八梯六十二户',
       '三梯二十一户', '四梯三十一户', '三梯三十三户', '四梯四十六户', '一梯二十二户', '四梯十二户', '两梯一户',
       '两梯二十三户', '三梯二十八户', '四梯三十五户', '四梯四十二户', '四梯十五户', '一梯十户', '三梯十九户',
       '两梯二十七户', '一梯二十四户', '五梯三十户', '两梯二十二户', '四梯四十四户', '三梯三十六户',

In [23]:
# 定义函数来提取电梯数和户数并计算比例
def calculate_ratio(row):
    if pd.isna(row):
        return np.nan
    elevator_chinese = row.split('梯')[0]
    elevator_num = cn2an(elevator_chinese, mode='smart')
    household_chinese = row.split('梯')[1].replace('户', '')
    household_num = cn2an(household_chinese, mode='smart')
    return elevator_num / household_num

In [24]:
# 应用函数计算比例
fullData['一户平均电梯数'] = fullData['梯户比例'].apply(calculate_ratio)

In [25]:
fullData['一户平均电梯数'] = fullData['一户平均电梯数'].apply(lambda x: 7 if pd.notna(x) and x > 7 else x)

In [26]:
fullData['一户平均电梯数'].fillna(0, inplace=True)

## 经纬度

In [7]:
trainData.plot(kind = "scatter", x = "lon", y = "lat", alpha = 0.1, figsize = (10,7), s = trainData["建筑面积"].apply(lambda x:x[:-1]).astype('float')/10, label = "建筑面积",c = "价格", cmap = plt.get_cmap("jet"), colorbar = True,edgecolors="k", linewidths=0.2, zorder=1)

plt.legend()

<Figure size 720x504 with 2 Axes>

In [29]:
import matplotlib.pyplot as plt

X = fullData[['lon', 'lat']].values

# 设置聚类数量为6
kmeans = KMeans(n_clusters=8, random_state=42)
# 拟合聚类模型
kmeans.fit(X)

# 获取每个样本的聚类标签
fullData['area_feature'] = kmeans.labels_ + 1

# 可视化聚类结果（可选步骤）
plt.scatter(X[:, 0], X[:, 1], c=fullData['area_feature'], cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='x', s=200)
plt.xlabel('lon')
plt.ylabel('lat')
plt.title('K - Means Clustering Result')
plt.show()

<Figure size 432x288 with 1 Axes>

## 文字描述处理 

In [27]:
#因为核心卖点与户型介绍有很多重复的部分，所以合并起来
fullData['卖点-户型'] = fullData['核心卖点'] + fullData['户型介绍']

In [28]:
#寻找高频词函数
def extract_high_frequency_words(df, column_name, top_n=10):
    # 将指定列的空值替换为空字符串
    df[column_name] = df[column_name].fillna('')

    # 定义一个函数用于分词
    def tokenize(text):
        # 使用正则表达式匹配中文词语
        words = re.findall(r'[\u4e00-\u9fa5]+', text)
        return words

    # 对指定列进行分词
    all_words = []
    for description in df[column_name]:
        words = tokenize(description)
        all_words.extend(words)

    # 统计词频
    word_counts = Counter(all_words)

    # 显示前 top_n 个高频词
    return word_counts.most_common(top_n)

In [29]:
extract_high_frequency_words(fullData,'卖点-户型', top_n=20)

[('户型方正', 4890),
 ('采光好', 3861),
 ('南北通透', 2743),
 ('采光充足', 2630),
 ('视野开阔', 1913),
 ('楼层好', 1777),
 ('精装修', 1541),
 ('中间楼层', 1490),
 ('全明户型', 1055),
 ('无遮挡', 907),
 ('视野好', 896),
 ('视野宽阔', 885),
 ('明厨明卫', 871),
 ('年', 844),
 ('一梯两户', 798),
 ('米', 770),
 ('平米', 766),
 ('交通便利', 753),
 ('无浪费面积', 740),
 ('此房满五唯一', 718)]

In [30]:
extract_high_frequency_words(fullData,'周边配套', top_n=33)

[('米', 11996),
 ('医院', 7818),
 ('市', 5656),
 ('公里', 5242),
 ('银行', 4895),
 ('建设银行', 4853),
 ('工商银行', 4291),
 ('商场', 3935),
 ('路', 3866),
 ('公园', 3633),
 ('中国银行', 3087),
 ('永辉', 2982),
 ('农业银行', 2863),
 ('号线', 2851),
 ('家乐福', 1930),
 ('沃尔玛', 1904),
 ('地铁', 1899),
 ('华润万家', 1877),
 ('购物', 1683),
 ('医疗', 1648),
 ('配套齐全', 1464),
 ('交通银行', 1423),
 ('电影院', 1301),
 ('生活便利', 1293),
 ('小区', 1276),
 ('出行方便', 1253),
 ('人人乐', 1220),
 ('距离', 1106),
 ('物美', 1103),
 ('交通便利', 1090),
 ('万达广场', 1079),
 ('交通', 1054),
 ('距离小区', 1042)]

In [31]:
extract_high_frequency_words(fullData,'交通出行', top_n=15)

[('路', 106104),
 ('米', 17841),
 ('号线', 16291),
 ('地铁', 8210),
 ('公里', 6989),
 ('出行方便', 5870),
 ('公交', 4749),
 ('交通便利', 4498),
 ('有', 4481),
 ('轻轨', 2593),
 ('距离', 2551),
 ('等', 2539),
 ('路等', 2170),
 ('出行便利', 2027),
 ('米左右', 1974)]

In [32]:
#计算高频词个数的函数
def count_contains_specified_words(df, column_name, specified_words, new_column_name, merged_expressions=None):
    counts = []
    if merged_expressions:
        # 提取所有统一标识
        unified_identifiers = list(merged_expressions.keys())
        # 合并统一标识到指定统计词列表
        all_specified_words = specified_words + unified_identifiers
    else:
        all_specified_words = specified_words

    for description in df[column_name]:

         # 将描述转换为字符串类型，处理可能的空值或非字符串类型
        description = str(description)

        # 定义需要合并的表述及其统一标识
        if merged_expressions:

            # 替换描述文本中的特定表述为统一标识
            for unified, expressions in merged_expressions.items():
                for exp in expressions:
                    description = description.replace(exp, unified)
        
        # 用于存储已匹配到的词，避免重复计数
        matched_words = set()
        for word in all_specified_words:
            if word in description:
                matched_words.add(word)
        
        # 将该描述中包含指定词的数目添加到计数列表中
        counts.append(len(matched_words))
    df[new_column_name] = counts
    return df

In [33]:
jt_sw = ['公交',"出行便利标识","有无地铁标识"]
jt_me = {
    "出行便利标识": ["出行方便", "交通便利"],
    "有无地铁标识": ["地铁","号线","轻轨"]
    
}
count_contains_specified_words(fullData,'交通出行',jt_sw,'交通出行_count',jt_me)

,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,建筑面积,套内面积,...,南,西,北,东南,西南,西北,东北,一户平均电梯数,卖点-户型,交通出行_count
0,0,79.0,111.0,二至三环,人定湖西里,6564200.0,2室1厅1厨1卫,中楼层 (共5层),52.30,NaN,...,1,0,1,0,0,0,0,0.333333,此房是南北通透小板楼，户型方正，格局合理房子是南北通透户型方正采光好，前后没有遮挡视野好，通...,0
1,0,43.0,231.0,五至六环,龙跃苑四区,4174000.0,3室1厅1厨1卫,顶层 (共6层),127.44,123.70,...,1,0,1,0,0,0,0,0.500000,南北通透商品房自住装修无个税房子三居一卫，户型方正，南北通透，客厅朝南带阳台，主卧朝南，东向...,0
2,0,97.0,54.0,五至六环,名都园,16310000.0,4室2厅1厨4卫,底层 (共3层),228.54,NaN,...,1,0,0,0,0,0,0,1.000000,有挑空客厅私家车库花园和*地上三层地下一层四期密度低没有公寓一层车库可以做卧室花园有100平米左右,0
3,0,62.0,568.0,三至四环,保利海德公园,2834600.0,2房间2卫,低楼层 (共10层),43.60,29.39,...,0,0,1,0,0,0,0,0.130435,,0
4,0,62.0,226.0,三至四环,京投银泰琨御府,1954000.0,1房间1卫,中楼层 (共10层),39.85,29.94,...,1,0,0,0,0,0,0,0.181818,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98914,6,71.0,244.0,无,横巷市委家属院南区,NaN,2室1厅1厨1卫,中楼层 (共7层),65.78,NaN,...,1,0,1,0,0,0,0,0.000000,,2
98915,6,91.0,723.0,无,润天小区,NaN,3室2厅1厨2卫,高楼层 (共6层),138.01,NaN,...,1,0,1,0,0,0,0,0.000000,,0
98916,6,83.0,380.0,三环外,御湖蓝湾,NaN,3室2厅1厨2卫,中楼层 (共18层),110.00,NaN,...,1,0,1,0,0,0,0,0.000000,户型方正，视野好，采光好，随时可签采光好，前后无遮挡，楼层佳，视野开阔，业主诚心出售，随时可...,0
98917,6,83.0,380.0,三环外,御湖蓝湾,NaN,3室2厅1厨2卫,高楼层 (共9层),136.00,NaN,...,1,0,1,0,0,0,0,0.000000,,0


In [34]:
zb_sw = ['银行','公园','电影院',"超市标识","就医标识","生活便利标识","商场标识"]
zb_me = {
    "超市标识": ['永辉', '家乐福','沃尔玛','华润万家','人人乐','物美'],
    "就医标识": ['医院','医疗'],
    "生活便利标识": ['配套齐全','生活便利','出行方便','交通便利'],
    "商场标识": ['商场','万达广场']
}
count_contains_specified_words(fullData,'周边配套',zb_sw,'周边配套_count',zb_me)

,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,建筑面积,套内面积,...,西,北,东南,西南,西北,东北,一户平均电梯数,卖点-户型,交通出行_count,周边配套_count
0,0,79.0,111.0,二至三环,人定湖西里,6564200.0,2室1厅1厨1卫,中楼层 (共5层),52.30,NaN,...,0,1,0,0,0,0,0.333333,此房是南北通透小板楼，户型方正，格局合理房子是南北通透户型方正采光好，前后没有遮挡视野好，通...,0,4
1,0,43.0,231.0,五至六环,龙跃苑四区,4174000.0,3室1厅1厨1卫,顶层 (共6层),127.44,123.70,...,0,1,0,0,0,0,0.500000,南北通透商品房自住装修无个税房子三居一卫，户型方正，南北通透，客厅朝南带阳台，主卧朝南，东向...,0,3
2,0,97.0,54.0,五至六环,名都园,16310000.0,4室2厅1厨4卫,底层 (共3层),228.54,NaN,...,0,0,0,0,0,0,1.000000,有挑空客厅私家车库花园和*地上三层地下一层四期密度低没有公寓一层车库可以做卧室花园有100平米左右,0,0
3,0,62.0,568.0,三至四环,保利海德公园,2834600.0,2房间2卫,低楼层 (共10层),43.60,29.39,...,0,1,0,0,0,0,0.130435,,0,0
4,0,62.0,226.0,三至四环,京投银泰琨御府,1954000.0,1房间1卫,中楼层 (共10层),39.85,29.94,...,0,0,0,0,0,0,0.181818,,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98914,6,71.0,244.0,无,横巷市委家属院南区,NaN,2室1厅1厨1卫,中楼层 (共7层),65.78,NaN,...,0,1,0,0,0,0,0.000000,,2,2
98915,6,91.0,723.0,无,润天小区,NaN,3室2厅1厨2卫,高楼层 (共6层),138.01,NaN,...,0,1,0,0,0,0,0.000000,,0,0
98916,6,83.0,380.0,三环外,御湖蓝湾,NaN,3室2厅1厨2卫,中楼层 (共18层),110.00,NaN,...,0,1,0,0,0,0,0.000000,户型方正，视野好，采光好，随时可签采光好，前后无遮挡，楼层佳，视野开阔，业主诚心出售，随时可...,0,0
98917,6,83.0,380.0,三环外,御湖蓝湾,NaN,3室2厅1厨2卫,高楼层 (共9层),136.00,NaN,...,0,1,0,0,0,0,0.000000,,0,0


In [35]:
md_sw = ['户型方正','南北通透','看房方便','业主诚心出售',"采光标识","楼层标识","视野标识","配套标识"]
md_me = {
    "采光标识": ['采光好', '采光充足'],
    "楼层标识": ['楼层好','中间楼层'],
    "视野标识": ['视野开阔','视野宽阔','视野好','无遮挡'],
    "配套标识": ['配套设施齐全','配套齐全']
}
count_contains_specified_words(fullData,'卖点-户型',md_sw,'卖点-户型_count',md_me)

,城市,区域,板块,环线,小区名称,价格,房屋户型,所在楼层,建筑面积,套内面积,...,北,东南,西南,西北,东北,一户平均电梯数,卖点-户型,交通出行_count,周边配套_count,卖点-户型_count
0,0,79.0,111.0,二至三环,人定湖西里,6564200.0,2室1厅1厨1卫,中楼层 (共5层),52.30,NaN,...,1,0,0,0,0,0.333333,此房是南北通透小板楼，户型方正，格局合理房子是南北通透户型方正采光好，前后没有遮挡视野好，通...,0,4,4
1,0,43.0,231.0,五至六环,龙跃苑四区,4174000.0,3室1厅1厨1卫,顶层 (共6层),127.44,123.70,...,1,0,0,0,0,0.500000,南北通透商品房自住装修无个税房子三居一卫，户型方正，南北通透，客厅朝南带阳台，主卧朝南，东向...,0,3,2
2,0,97.0,54.0,五至六环,名都园,16310000.0,4室2厅1厨4卫,底层 (共3层),228.54,NaN,...,0,0,0,0,0,1.000000,有挑空客厅私家车库花园和*地上三层地下一层四期密度低没有公寓一层车库可以做卧室花园有100平米左右,0,0,0
3,0,62.0,568.0,三至四环,保利海德公园,2834600.0,2房间2卫,低楼层 (共10层),43.60,29.39,...,1,0,0,0,0,0.130435,,0,0,0
4,0,62.0,226.0,三至四环,京投银泰琨御府,1954000.0,1房间1卫,中楼层 (共10层),39.85,29.94,...,0,0,0,0,0,0.181818,,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98914,6,71.0,244.0,无,横巷市委家属院南区,NaN,2室1厅1厨1卫,中楼层 (共7层),65.78,NaN,...,1,0,0,0,0,0.000000,,2,2,0
98915,6,91.0,723.0,无,润天小区,NaN,3室2厅1厨2卫,高楼层 (共6层),138.01,NaN,...,1,0,0,0,0,0.000000,,0,0,0
98916,6,83.0,380.0,三环外,御湖蓝湾,NaN,3室2厅1厨2卫,中楼层 (共18层),110.00,NaN,...,1,0,0,0,0,0.000000,户型方正，视野好，采光好，随时可签采光好，前后无遮挡，楼层佳，视野开阔，业主诚心出售，随时可...,0,0,4
98917,6,83.0,380.0,三环外,御湖蓝湾,NaN,3室2厅1厨2卫,高楼层 (共9层),136.00,NaN,...,1,0,0,0,0,0.000000,,0,0,0


In [36]:
fullData.drop(['核心卖点', '户型介绍','周边配套','交通出行'], axis=1, inplace=True)

## dummy

In [37]:
# 定义一个函数用于使用“众数”替换空值
def fill_missing_with_mode(df, column):
    # 计算该列出现次数最多的值
    mode_value = df.loc[:84132, column].mode().values[0]
    # 将该列的空值替换为出现次数最多的值
    df[column] = df[column].fillna(mode_value)
    return df

In [38]:
# 对建筑结构列进行众数替换
fullData = fill_missing_with_mode(fullData, '建筑结构')
fullData = fill_missing_with_mode(fullData, '房屋年限')

In [39]:
fullData['装修情况'].fillna("简装", inplace=True)

In [40]:
fullData['交易时间'] = pd.to_datetime(fullData['交易时间'], errors='coerce')
fullData['上次交易'] = pd.to_datetime(fullData['上次交易'], errors='coerce')

mode_value = fullData.loc[:84132, '上次交易'].mode()[0]
fullData['上次交易'].fillna(mode_value, inplace=True)

In [41]:
fullData['上次交易间隔'] = (fullData['交易时间'] - fullData['上次交易']).dt.days

In [42]:
fullData['配备电梯'] = fullData['配备电梯'].fillna('无')
fullData['别墅类型'] = fullData['别墅类型'].fillna('无')

In [43]:
#train之前drop的数据
fullData.drop(['area_block_combined','区域','板块','小区名称','房屋户型','所在楼层','房屋朝向','梯户比例','上次交易','交易时间','卖点-户型','房屋优势','匹配小区','套内面积','交易权属'], axis=1, inplace=True)

In [44]:
columns_to_dummy = ['城市', '环线','建筑结构','装修情况','配备电梯','别墅类型','房屋用途','房屋年限','产权所属','年份','cluster_label']

# 对指定列进行独热编码
fullData = pd.get_dummies(fullData, columns=columns_to_dummy,drop_first=True)


In [45]:
fullData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98919 entries, 0 to 98918
Columns: 108 entries, 价格 to cluster_label_12
dtypes: float64(13), int64(20), uint8(75)
memory usage: 34.8 MB


In [45]:
fullData.to_csv('/home/mw/project/fullData.csv', index=True)